In [ ]:
%reload_ext rich

import mitsuba as mi

import eradiate
import eradiate.experiments._low_level as ll
from eradiate import unit_registry as ureg
from eradiate.contexts import KernelContext
from eradiate.kernel._kernel_dict_new import (
    KernelDictionary,
    KernelSceneParameterFlag,
    KernelSceneParameterMap,
    mi_render,
    scene_parameter,
)

eradiate.set_mode("mono")

In [ ]:
display(KernelDictionary())
display(KernelSceneParameterMap())

In [ ]:
@scene_parameter(flags=KernelSceneParameterFlag.SPECTRAL)
def spectrum_lambertian(ctx: KernelContext):
    if ctx.si.w == 440.0 * ureg.nm:
        return 0.1
    elif ctx.si.w == 550.0 * ureg.nm:
        return 0.5
    elif ctx.si.w == 660.0 * ureg.nm:
        return 0.9
    else:
        return 0.0


exp = ll.CoreExperiment(
    kdict={
        "bsdf_surface": {"type": "diffuse", "reflectance": 0.5, "id": "lambertian"},
        "surface": {"type": "rectangle", "bsdf": {"type": "ref", "id": "lambertian"}},
        "illumination": {"type": "directional", "direction": [0, 0, -1]},
        "camera": {
            "type": "perspective",
            "to_world": mi.ScalarTransform4f.look_at(
                origin=[3, 3, 3],
                target=[0, 0, 0],
                up=[0, 0, 1],
            ),
        },
        "integrator": {"type": "path"},
    },
    kpmap={"lambertian.reflectance.value": spectrum_lambertian},
)
display(exp)
display(exp.mi_scene)
mi_params = exp.mi_params
print(mi_params)

In [ ]:
kpmap = exp.kpmap_full().render(KernelContext({"w": 660.0}))
display(kpmap)
mi_params.update(kpmap)
mi_params

In [ ]:
import mitsuba as mi

mi.Bitmap(mi.render(exp.mi_scene))

In [ ]:
ctxs = [KernelContext({"w": w}) for w in [440.0, 550.0, 660.0]]
display(ctxs)
mi_render(exp.mi_scene, exp.mi_params, exp.kpmap_full(), ctxs)